# Tutorial 2: Stop detection in trajectories

This notebook shows how to process device-level trajectory data, in different formats, to detect stops using ```nomad```. Stop detection is an important step in
pre-processing trajectory data and in making sense of trajectories by grouping together pings that reflect stationary behavior. The output of stop-detection algorithms is commonly a "stop table", indicating when a stop started, its duration, and a pair of coordinates that approximates the location of the group of pings (typically the centroid). Alternatively, ```nomad``` allows users to retrieve a cluster label for each ping (useful for plotting, for example)

In [ ]:
import importlib
import pandas as pd
import matplotlib.pyplot as plt
import nomad.filters as filters
import nomad.daphmeIO as loader
import nomad.constants as constants
from nomad.stop_detection import ta_dbscan
from nomad.stop_detection import lachesis

In [ ]:
import pandas as pd
from scipy.spatial.distance import pdist, cdist
import numpy as np
import math
import datetime as dt
from datetime import timedelta
import matplotlib.pyplot as plt
import itertools
from collections import defaultdict
import sys
import os
import pdb

## Load data sample

For these examples we load some test data from ```nomad``` which has the following trajectory columns (notice that ```traj_cols``` maps the data's column names to default names used by ```nomad```)

In [ ]:
traj_cols = {'user_id':'user_id',
             'latitude':'dev_lat',
             'longitude':'dev_lon',
             'datetime':'local_datetime'}

In [ ]:
path = '../data/sample4/'
data = loader.from_file(path, traj_cols=traj_cols, format='csv')

This synthetic data has records for 100 users for a 1 week period, with spherical coordinates (lat, lon) and datetime format for the time component of each ping. 

In [ ]:
data.head()

## Stop detection algorithms

The stop detection algorithms in ```nomad``` are applied to each user's trajectories separately. Thus, we demonstrate first by sampling a single user's data.

In [ ]:
user_sample = data.loc[data.user_id == "angry_spence"]

In [ ]:
user_sample.head()

For this user, the trajectory data has 1696 rows (pings) and covers a period of 15 days (start date: 2024-01-01, end date: 2024-01-15). We can visualize the trajectory for one day below: 

In [ ]:
plot_data = user_sample[user_sample['local_datetime'].dt.date == pd.to_datetime('2024-01-01').date()]

fig, ax = plt.subplots(figsize=(9, 5))

ax.scatter(x=plot_data['dev_lon'], 
           y=plot_data['dev_lat'],
           s=6,
           color='black',
           alpha=1,
           zorder=2)

ax.plot(plot_data['dev_lon'],
        plot_data['dev_lat'],
        linewidth=1, color='blue', alpha=0.2)

ax.set_yticklabels([])
ax.set_xticklabels([])
ax.set_xticks([])
ax.set_yticks([])

plt.tight_layout()
plt.show()

### Sequential stop detection (Lachesis)

The first stop detection algorithm implemented in ```nomad``` is a sequential algorithm described in "Project Lachesis: Parsing and Modeling Location Histories". This algorithm for extracting stays is dependent on two parameters: the roaming distance and the stay duration. Roaming distance represents the maximum distance an object can move away from a point location and still be considered to be staying at that location. Stay duration is the minimum amount of time an object must spend within the roaming distance of a location to qualify as a stay. The algorithm identifies contiguous sequences of raw location data points that stay within the roaming distance for at least the duration of the stay duration

This algorithm has the following parameters, which determine the size of the resulting stops:
* ```dur_min```: Minimum duration for a stay in minutes.
* ```dt_max```: Maximum time gap permitted between consecutive pings in a stay in minutes (dt_max should be greater than dur_min).
* ```delta_roam```: Maximum roaming distance for a stay in meters.

In [ ]:
DUR_MIN = 60
DT_MAX = 120
DELTA_ROAM = 50

The Lachesis algorithm can output a complete table of attributes for identified stops, including the start time, end time, the medoid coordinates, duration, number of pings in the stop, and diameter.

In [ ]:
%%time
Lachesis.lachesis(traj=user_sample, dur_min=DUR_MIN, dt_max=DT_MAX, delta_roam=DELTA_ROAM, traj_cols=traj_cols, complete_output=True).head()

An additional argument, ```complete_output```, can be passed to only output the stop start time, duration, and medoid coordinates.

In [ ]:
%%time
Lachesis.lachesis(traj=user_sample, dur_min=DUR_MIN, dt_max=DT_MAX, delta_roam=DELTA_ROAM, traj_cols=traj_cols, complete_output=False).head()

We can also get the final cluster label for each of the pings, including those who were identified as noise.

In [ ]:
%%time
Lachesis._lachesis_labels(traj=user_sample, dur_min=DUR_MIN, dt_max=DT_MAX, delta_roam=DELTA_ROAM, traj_cols=traj_cols).head()

The data could also come with different formats for spatial and temporal variables, the algorithm can handle those situations as well.

In [ ]:
%%time
# x, y, timestamp (unix time)
traj_cols_x_y_timestamp = {'user_id':'user_id',
                          'x':'x',
                          'y':'y',
                          'timestamp':'unix_time'}

user_sample_x_y_timestamp = user_sample.copy()
user_sample_x_y_timestamp = filters.to_projection(user_sample_x_y_timestamp, longitude='dev_lon', latitude='dev_lat')
user_sample_x_y_timestamp['unix_time'] = user_sample_x_y_timestamp['local_datetime'].astype('int64') // 10**9

# Lachesis with x, y, and timestamp (unix time)
Lachesis.lachesis(user_sample_x_y_timestamp, DUR_MIN, DT_MAX, DELTA_ROAM, traj_cols=traj_cols_x_y_timestamp, complete_output=False).head()

### Density based stop detection (Temporal DBSCAN)

The second stop detection algorithm implemented in ```nomad``` is a density-based algorithm with time-constraints, Temporal DBSCAN. 

This algorithm for clustering user pings combines temporal and spatial dimensions, relying on three key parameters: the time threshold, the distance threshold, and the minimum number of points. The time threshold defines the maximum time difference (in minutes) between two consecutive pings for them to be considered neighbors within the same cluster. The distance threshold specifies the maximum spatial distance (in meters) between two pings for them to be considered neighbors. The minimum points parameter sets the minimum number of points required for a dense region to form a cluster. If a region contains fewer than minimum number of points required, it is treated as noise. The algorithm identifies clusters by grouping contiguous pings that meet both the temporal and spatial criteria, while also ensuring that each cluster has enough density to be considered valid.

This algorithm has the following parameters, which determine the size of the resulting stops:
* ```time_thresh```: Time threshold in minutes for identifying neighbors.
* ```dist_thresh```: Distance threshold in meters for identifying neighbors.
* ```min_pts```: Minimum number of points required to form a dense region (core point).

In [ ]:
TIME_THRESH = 100
DIST_THRESH = 40
MIN_PTS = 10

Similarly to Lachesis, the Temporal DBSCAN algorithm can output a complete table of attributes for identified stops, including the start time, end time, the medoid coordinates, duration, number of pings in the stop, and diameter.

In [ ]:
%%time
DBSCAN.temporal_dbscan(user_sample, TIME_THRESH, DIST_THRESH, MIN_PTS, traj_cols=traj_cols, complete_output=True).head()

The additional argument ```complete_output``` can also be passed to only output the stop start time, duration, and medoid coordinates.

In [ ]:
%%time
DBSCAN.temporal_dbscan(user_sample, TIME_THRESH, DIST_THRESH, MIN_PTS, traj_cols=traj_cols, complete_output=False).head()

We can also get the final cluster and core labels for each of the pings.

In [ ]:
%%time
DBSCAN._temporal_dbscan_labels(user_sample, TIME_THRESH, DIST_THRESH, MIN_PTS, traj_cols=traj_cols).head()

The Temporal DBSCAN algorithm also handles data that comes with different formats for spatial and temporal variables.

In [ ]:
%%time
DBSCAN.temporal_dbscan(user_sample_x_y_timestamp, TIME_THRESH, DIST_THRESH, MIN_PTS, traj_cols=traj_cols_x_y_timestamp, complete_output=False).head()